In [1]:
# Written by Jesse van Duijne on 13-05-2020

In [ ]:
# This script augments images in different types
# ..which means the same images will be multiplied

In [5]:
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import cv2
import random
from skimage.transform import warp, AffineTransform
%matplotlib inline

In [6]:
path = "C:\\Users\\Jesse\\Hogeschool Inholland\\Minor Big Data - Group - General\\Opdracht\\Dataset\\v2"
input_folder = "Good v0.2 (own input - resized)"
input_folder_path = os.path.join(path, input_folder)
output_folder = "Good v0.3 (own input - augmented)"
output_folder_path = os.path.join(path, output_folder)
file_sign = 'go'  # File name to point them out

In [12]:
for filename in os.listdir(input_folder_path):
    if filename.endswith('.jpg'):
        imgpath = os.path.join(input_folder_path, filename)
        filename_stripped = 'g' + filename.strip('.jpg')             
        output_file_path_typeless = os.path.join(output_folder_path, filename_stripped)        
        
        # 1 Read default img
        img = imageio.imread(imgpath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(output_file_path_typeless + "_default.jpg", img) 
        
        # 2 Rotate
        rotate=iaa.Affine(rotate=(-50, 30))
        rotated_image=rotate.augment_image(img)        
        cv2.imwrite(output_file_path_typeless + "_rotated.jpg", rotated_image)
        
        # 3 Gauss noise
        gaussian_noise=iaa.AdditiveGaussianNoise(10,20)
        noise_image=gaussian_noise.augment_image(img)
        cv2.imwrite(output_file_path_typeless + "_gauss_noise.jpg", noise_image)
        
        # 4 Cropping
        crop = iaa.Crop(percent=(0, 0.3)) # crop image
        crop_image = crop.augment_image(img)
        cv2.imwrite(output_file_path_typeless + "_crop.jpg", crop_image)
        
        # 5 Shearing
        shear = iaa.Affine(shear=(0,40))
        shear_image = shear.augment_image(img)
        cv2.imwrite(output_file_path_typeless + "_shear.jpg", shear_image)
        
        # 6 Flipping horizontally
        flip_hr = iaa.Fliplr(p=1.0)
        flip_hr_image = flip_hr.augment_image(img)
        cv2.imwrite(output_file_path_typeless + "_flip_hr.jpg", flip_hr_image)
        
        # 7 Flip vertically
        flip_vr = iaa.Flipud(p=1.0)
        flip_vr_image= flip_vr.augment_image(img)
        cv2.imwrite(output_file_path_typeless + "_flip_vr.jpg", flip_vr_image)
        
        # 8 Change brightness
        contrast = iaa.GammaContrast(gamma=2.0)
        contrast_image = contrast.augment_image(img)
        cv2.imwrite(output_file_path_typeless + "_constrast.jpg", contrast_image)
        
        # 9 Scaling image
        scale_im = iaa.Affine(scale={"x": (1.5, 1.0), "y": (1.5, 1.0)})
        scale_image = scale_im.augment_image(img)
        cv2.imwrite(output_file_path_typeless + "_scaled.jpg", scale_image)
                        
        # 10 Blur image
        blurred_img = cv2.blur(img, (3, 3))
        cv2.imwrite(output_file_path_typeless + "_blurred.jpg", blurred_img)
            
        # 11 Salt & Pepper noise
        sp_image = add_sp_noise(img, 0.001)
        cv2.imwrite(output_file_path_typeless + "_saltpepper.jpg", sp_image)
    
        
        

In [8]:
def add_sp_noise(img, prob):
    output = np.zeros(img.shape, np.uint8)
    thres = 1 - prob
    for i in range(img.shape[0]):
        for j  in range(img.shape[1]):
            rnd = random.random()
            if rnd < prob:
                output[i][j] = 0
            elif rnd > thres:
                output[i][j] = 255
            else: 
                output[i][j] = img[i][j]
    return output

In [9]:
def shift_image(X, dx, dy):
    X = np.roll(X, dy, axis=0)
    X = np.roll(X, dx, axis=1)
    if dy>0:
        X[:dy, :] = 0
    elif dy<0:
        X[dy:, :] = 0
    if dx>0:
        X[:, :dx] = 0
    elif dx<0:
        X[:, dx:] = 0
    return X